# Projet Data Analyse - Recrutement de jeunes attaquants

Ce notebook présente la phase de nettoyage et préparation des données à partir de deux jeux de données :
- `all_fifa_players.csv` (provenant de FIFA)
- `players.csv` (provenant de Transfermarkt)

Objectif : créer une base propre et pertinente pour construire un dashboard Power BI permettant d'identifier des jeunes attaquants (<=21 ans) à fort potentiel et à faible valeur marchande (<= 1000000€)


## Étape 1 : Exploration des données

In [37]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from datetime import datetime

# Chargement des fichiers CSV dans des DataFrames pandas
fifa = pd.read_csv('all_fifa_players.csv')
transfermarkt = pd.read_csv('players.csv')

print(f"Nombre de joueurs FIFA chargés : {fifa.shape[0]}")
print(f"Nombre de joueurs Transfermarkt chargés : {transfermarkt.shape[0]}")


Nombre de joueurs FIFA chargés : 37158
Nombre de joueurs Transfermarkt chargés : 32601


In [ ]:
fifa.head()

In [ ]:
fifa.info()

In [ ]:
transfermarkt.head()

In [ ]:
transfermarkt.info()

In [ ]:
#visualisation des colonnes des deux dataframes
print(fifa.columns)
print(transfermarkt.columns)

#### Sélection des colonnes utiles

Les datasets contiennent de nombreuses colonnes. On conserve uniquement celles pertinentes pour l'analyse afin d'alléger la mémoire et simplifier le traitement.


In [38]:
fifa_colonnes_pertinentes = ['Player', 'League', 'Contract End', 'Age', 'Potential Score', 
                             'Player ID', 'Height', 'Weight', 'Preferred Foot', 'Best Overall', 
                             'Best Position', 'Value', 'Wage', 'Release Clause', 'Finishing', 
                             'Heading Accuracy', 'Short Passing', 'Volleys', 'Dribbling', 'Ball Control', 
                             'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 
                             'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Aggression',
                             'Attack Position', 'Vision', 'Composure', 'Weak Foot', 'Skill Moves'] 

transfermarkt_colonnes_pertinentes = ['player_id','name', 'date_of_birth','sub_position',
'current_club_name', 'market_value_in_eur']

# Extraction des colonnes dans des copies indépendantes afin d'éviter d'impacter le dataframe de base
fifa_clean = fifa[fifa_colonnes_pertinentes].copy()
transfermarkt_clean = transfermarkt[transfermarkt_colonnes_pertinentes].copy()

## Étape 2 : Nettoyage des données FIFA

#### Renommage des colonnes

On renomme les colonnes pour :
- homogénéiser la nomenclature (snake_case)
- clarifier les noms (par ex: en ajoutant des préfixes pour éviter les confusions)
- faciliter la manipulation et la lecture du code

In [ ]:
fifa_clean = fifa_clean.rename(columns={
    'Player': 'player_name',
    'Player ID': 'fifa_player_id',
    'Age': 'age',
    'Height': 'height_cm', 
    'Weight': 'weight_kg',
    'League': 'league',
    'Best Position': 'fifa_position',
    'Potential Score': 'potential_score',
    'Best Overall': 'overall_score',
    'Preferred Foot': 'preferred_foot',
    'Release Clause': 'release_clause',
    'Finishing': 'finishing',
    'Heading Accuracy': 'heading_accuracy',
    'Short Passing': 'short_passing',
    'Volleys': 'volleys',
    'Dribbling': 'dribbling',
    'Ball Control': 'ball_control',
    'Acceleration': 'acceleration',
    'Sprint Speed': 'sprint_speed',
    'Agility': 'agility',
    'Reactions': 'reactions',
    'Balance': 'balance',
    'Shot Power': 'shot_power',
    'Jumping': 'jumping',
    'Stamina': 'stamina',
    'Strength': 'strength',
    'Long Shots': 'long_shots',
    'Aggression': 'aggression',
    'Attack Position': 'attack_position',
    'Vision': 'vision',
    'Composure': 'composure',
    'Weak Foot': 'weak_foot',
    'Skill Moves': 'skill_moves',
    'Value': 'value', 
    'Wage': 'wage',
    'Contract End': 'contract_end'
})

transfermarkt_clean = transfermarkt_clean.rename(columns={
    'player_id': 'tm_player_id',
    'name': 'player_name',
    'sub_position': 'tm_position',
    'current_club_name': 'club_name',
    'market_value_in_eur': 'market_value_eur'
})

# Affiche les nouvelles colonnes
print("Nouvelles colonnes FIFA:", fifa_clean.columns)
print("Nouvelles colonnes Transfermarkt:", transfermarkt_clean.columns)

#### Nettoyage des noms des joueurs

L'ID (fifa_player_id et tm_player_id) n'est pas une clé de jointure fiable entre les deux datasets car ils proviennent de sources différentes. La clé de jointure principale sera donc le nom du joueur. Pour maximiser les chances de succès de la jointure, il faut nettoyer cette colonne player_name dans les deux dataframes.

Afin de faciliter la fusion des deux datasets, on standardise les noms des joueurs en :
- minuscules
- suppression des accents
- suppression des points et tirets

Cette étape vise à harmoniser les noms malgré les différences de formatage entre les sources.

In [40]:
#fonction pour uniformiser les noms
def clean_player_name(name):
    if pd.isna(name):
        return name
    name = unidecode(str(name).strip().lower())
    name = name.replace('.', '').replace('-', ' ')
    return name

fifa_clean['player_name_cleaned'] = fifa_clean['player_name'].apply(clean_player_name)
transfermarkt_clean['player_name_cleaned'] = transfermarkt_clean['player_name'].apply(clean_player_name)

#### Conversion des valeurs monétaires (FIFA)

Les colonnes financières de FIFA contiennent des valeurs sous forme de chaînes avec des suffixes ('M' pour millions, 'K' pour milliers).
On convertit ces valeurs en float représentant le montant en euros.

In [41]:
def clean_currency(value):
    if pd.isna(value) or value == '':
        return None
    value = str(value).strip().replace('€', '')
    try:
        if 'M' in value:
            return float(value.replace('M', '')) * 1_000_000
        elif 'K' in value:
            return float(value.replace('K', '')) * 1_000
        else:
            return float(value)
    except ValueError:
        return None

fifa_clean['value'] = fifa_clean['value'].apply(clean_currency)
fifa_clean['wage'] = fifa_clean['wage'].apply(clean_currency)
fifa_clean['release_clause'] = fifa_clean['release_clause'].apply(clean_currency)

# on s’assure que cette colonne est bien numérique pour pouvoir faire des calculs
transfermarkt_clean['market_value_eur'] = pd.to_numeric(transfermarkt_clean['market_value_eur'], errors='coerce')

#### Vérification des types des colonnes financières

On s'assure que les colonnes monétaires ont bien été converties en types numériques, nécessaires pour les analyses et calculs.

In [ ]:
print("\nTypes de données des colonnes financières après nettoyage (FIFA):")
print(fifa_clean[['value', 'wage', 'release_clause']].info())
print("\nType de données de la colonne financière (Transfermarkt):")
print(transfermarkt_clean['market_value_eur'].info())

#### Conversion des colonnes dates

Pour faciliter les calculs d'âges et de durée des contrats, on convertit les colonnes date en datetime.

In [ ]:
fifa_clean['contract_end'] = pd.to_datetime(fifa_clean['contract_end'], errors='coerce')
transfermarkt_clean['date_of_birth'] = pd.to_datetime(transfermarkt_clean['date_of_birth'], errors='coerce')

today = pd.to_datetime("today")
transfermarkt_clean['age_calculated'] = (today - transfermarkt_clean['date_of_birth']).dt.days / 365.25

print("\nTypes de données des colonnes de date après nettoyage:")
print(fifa_clean['contract_end'].info())
print(transfermarkt_clean['date_of_birth'].info())
print(transfermarkt_clean['age_calculated'].info())

#### Aperçu des valeurs manquantes

In [ ]:
fifa_clean.isnull().sum().sort_values(ascending=False)

In [ ]:
transfermarkt_clean.isnull().sum().sort_values(ascending=False)

#### Suppression des lignes avec données critiques manquantes (Transfermarkt)

Les colonnes position, âge calculé et valeur de marché sont essentielles à la fusion et à l'analyse.

In [43]:
transfermarkt_clean.dropna(subset=['tm_position', 'age_calculated', 'market_value_eur'], inplace=True)

#### Extraction des valeurs numériques

Certaines colonnes FIFA peuvent contenir des valeurs au format texte. On extrait la partie numérique pour les convertir en int.

In [44]:
# fonction d'extraction 
def extract_numerique(df, columns):
    for col in columns:
        df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float).astype('Int64')
    return df

cols_to_clean = ['overall_score', 'potential_score', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'attack_position', 'vision', 'composure', 'height_cm', 'weight_kg']
fifa_clean = extract_numerique(fifa_clean, cols_to_clean)

#### Suppression de la colonne 'player_name' originale qui n'est plus utile

In [ ]:
fifa_clean.drop('player_name', axis=1, inplace=True)
transfermarkt_clean.drop('player_name', axis=1, inplace=True)

print("Colonnes restantes FIFA:", fifa_clean.columns)
print("Colonnes restantes Transfermarkt:", transfermarkt_clean.columns)

#### Suppression des doublons en gardant la première occurrence de chaque 'fifa_player_id' 

In [ ]:
fifa_clean.drop_duplicates(subset=['fifa_player_id'], keep='first', inplace=True)
print("Nombre de lignes après suppression des doublons (basé sur fifa_player_id) dans FIFA:", fifa_clean.shape[0])

# Vérifier les doublons dans Transfermarkt également, basés sur 'tm_player_id'
print("\nNombre de lignes avant suppression des doublons dans Transfermarkt:", transfermarkt_clean.shape[0])
print("Nombre de doublons basés sur 'tm_player_id':", transfermarkt_clean.duplicated(subset=['tm_player_id']).sum())

# Supprimer les doublons basés sur 'tm_player_id' dans Transfermarkt
transfermarkt_clean.drop_duplicates(subset=['tm_player_id'], keep='first', inplace=True)
print("Nombre de lignes après suppression des doublons (basé sur tm_player_id) dans Transfermarkt:", transfermarkt_clean.shape[0])

## Étape 3 : Fusion des données FIFA et Transfermarkt

In [47]:
merged_df = pd.merge(fifa_clean, transfermarkt_clean, how='inner', on='player_name_cleaned', suffixes=('_fifa', '_tm'))
print("Taille du dataset fusionné :", merged_df.shape)
merged_df.head(3)

Taille du dataset fusionné : (7329, 42)


,league,contract_end,age,potential_score,fifa_player_id,height_cm,weight_kg,preferred_foot,overall_score,fifa_position,...,composure,weak_foot,skill_moves,player_name_cleaned,tm_player_id,date_of_birth,tm_position,club_name,market_value_eur,age_calculated
0,1. Division (Cyprus),2027-01-01,34,73,193336,193,84,Right,73,GK,...,70,3,1,vid belec,73090,1990-06-06,Goalkeeper,US Salernitana 1919,500000.0,35.019849
1,1. Division (Cyprus),2026-01-01,37,73,194209,182,84,Right,73,ST,...,69,4,3,youssef el arabi,96533,1987-02-03,Centre-Forward,Olympiakos Syndesmos Filathlon Peiraios,300000.0,38.357290
2,1. Division (Cyprus),2026-01-01,28,72,212150,173,68,Right,72,CAM,...,77,4,4,max meyer,146164,1995-09-18,Attacking Midfield,Fenerbahçe Spor Kulübü,3000000.0,29.735797


#### Suppression des lignes avec données critiques manquantes

La colonne market_value_eur est essentielle à la fusion et à l'analyse. On ne garde donc que les joueurs qui ont une valeur marchande.

In [50]:
df_final = merged_df.dropna(subset=['market_value_eur']).copy()

print(f"Nombre de joueurs dans le DataFrame final (avec valeurs marchandes): {df_final.shape[0]}")
print("\nQuelques premières lignes du DataFrame final :")
print(df_final.head())

Nombre de joueurs dans le DataFrame final (avec valeurs marchandes): 7329

Quelques premières lignes du DataFrame final :
                 league contract_end  age  potential_score  fifa_player_id  \
0  1. Division (Cyprus)   2027-01-01   34               73          193336   
1  1. Division (Cyprus)   2026-01-01   37               73          194209   
2  1. Division (Cyprus)   2026-01-01   28               72          212150   
3  1. Division (Cyprus)   2025-01-01   29               71          227766   
4  1. Division (Cyprus)   2027-01-01   31               71          234774   

   height_cm  weight_kg preferred_foot  overall_score fifa_position  ...  \
0        193         84          Right             73            GK  ...   
1        182         84          Right             73            ST  ...   
2        173         68          Right             72           CAM  ...   
3        191         80           Left             71            CB  ...   
4        186         82      

#### Reorganisation des colonnes du dataframe final

In [ ]:
df_final = df_final[['player_name_cleaned', 'date_of_birth', 'age', 'age_calculated', 'height_cm', 'weight_kg', 'preferred_foot',
    'fifa_player_id', 'tm_player_id','fifa_position', 'tm_position','club_name', 'league', 'contract_end',
    'market_value_eur', 'value', 'wage', 'release_clause','overall_score', 'potential_score','finishing', 
    'heading_accuracy', 'short_passing', 'volleys', 'long_shots', 'shot_power',
    'dribbling', 'ball_control', 'vision', 'composure','acceleration', 'sprint_speed', 'agility', 'reactions', 
    'balance', 'jumping', 'stamina', 'strength','attack_position', 'aggression',
    'weak_foot']]

## Etape 4 : Export en fichier csv 

Fichier nettoyé prêt à être utilisé dans Power BI

In [51]:
df_final.to_csv('players_data_cleaned_for_powerbi.csv', index=False)
print("\nDataFrame final exporté vers 'players_data_cleaned_for_powerbi.csv'")


DataFrame final exporté vers 'players_data_cleaned_for_powerbi.csv'
